In [1]:
import random
import math
import numpy as np
import pandas as pd

## describe an exampalary problem

suppose we have 3 jobs and 2 machines:

In [2]:
jobs_ops=pd.read_excel("jobs_ops.xlsx")
jobs_ops

,Job_Name,Job,Operation,Mark,Machine,Duration,Description
0,Job1,1,1,O11,1,2,operation 1 for job 1
1,Job1,1,2,O12,2,2,operation 2 for job 1
2,Job 2,2,1,O21,2,3,operation 1 for job 2
3,Job 2,2,2,O22,1,1,operation 2 for job 2
4,Job 3,3,1,O31,2,1,operation 1 for job 3
5,Job 3,3,2,O32,1,1,operation 2 for job 3


In [7]:
jobs_N=3
machine_M=2

we want to minimize the total operation time. i
.e the completion time of the last operation

one feasible solution is this:<br>
( O11,O21,O22, O31,O32,O12  )
<br>
and the makespan is 6 minutes. <br>
but it's not neccesery the best solution

In [9]:
class Operation:
    def __init__(self,job_num,op_num):
        self.job=job_num
        self.op_num=op_num
        self.machine=int(jobs_ops.loc[(jobs_ops["Job"]==self.job) & (jobs_ops["Operation"]==self.op_num),"Machine"])
        self.duration=int(jobs_ops.loc[(jobs_ops["Job"]==self.job) & (jobs_ops["Operation"]==self.op_num),"Duration"])
        self.C=0 #the time when the operation has ended
        
    def __str__(self):
        return "Job:{}  Op:{}  Machine:{}  Duration: {}   C: {} ".format(self.job,self.op_num,self.machine,self.duration,self.C)

In [4]:
list_A=[1.3, 0.7, 2.4, 1.1, 3.4, 5.3]
list_A

[1.3, 0.7, 2.4, 1.1, 3.4, 5.3]

In [5]:
list_A_con=[(sorted(list_A).index(element)+1) for element in list_A]
list_A_con

[3, 1, 4, 2, 5, 6]

In [8]:
order_jobs=[element%jobs_N +1 for element in list_A_con]
order_jobs

[1, 2, 2, 3, 3, 1]

In [10]:
#Fixed sequence for testing

operations_seq=list()
for i in range(1,jobs_N+1):
    op_number=1
    for j in range(len(order_jobs)):
        if i==order_jobs[j]: 
            operations_seq.insert(j,Operation(i,op_number))
            op_number=op_number+1
for op in operations_seq:
    print(op)
    # NOW WE HAVE A LIST OF ALL OPERATIONS ! WE CAN CALCULATE THE "MAKESPAN"- i.e - the total duration of the sequence!

Job:1  Op:1  Machine:1  Duration: 2   C: 0 
Job:2  Op:1  Machine:2  Duration: 3   C: 0 
Job:2  Op:2  Machine:1  Duration: 1   C: 0 
Job:3  Op:1  Machine:2  Duration: 1   C: 0 
Job:3  Op:2  Machine:1  Duration: 1   C: 0 
Job:1  Op:2  Machine:2  Duration: 2   C: 0 


In [14]:
#General random sequence

list_B=list(np.random.uniform(1,6,6))
list_B_con=[(sorted(list_B).index(element)+1) for element in list_B]
list_B_con
order_jobs_B=[element%jobs_N +1 for element in list_B_con]
order_jobs_B
operations_seq=list()
for i in range(1,jobs_N+1):
    op_number=1
    for j in range(len(order_jobs_B)):
        if i==order_jobs_B[j]: 
            operations_seq.insert(j,Operation(i,op_number))
            op_number=op_number+1
for op in operations_seq:
    print(op)
# NOW WE HAVE A LIST OF ALL OPERATIONS ! WE CAN CALCULATE THE "MAKESPAN"- i.e - the total duration of the sequence!

Job:3  Op:1  Machine:2  Duration: 1   C: 0 
Job:3  Op:2  Machine:1  Duration: 1   C: 0 
Job:1  Op:1  Machine:1  Duration: 2   C: 0 
Job:1  Op:2  Machine:2  Duration: 2   C: 0 
Job:2  Op:1  Machine:2  Duration: 3   C: 0 
Job:2  Op:2  Machine:1  Duration: 1   C: 0 


In [15]:
def retrieve_last_operation(current_op,operations_seq):
    rev_seq=operations_seq[::-1].copy()
    for op in rev_seq:
        if op.job==current_op.job:
            return op

In [16]:
machines=list()
machines_jobs=list()
for m in range(machine_M):
    machines.append(list())
    machines_jobs.append(list())

In [17]:
for op in operations_seq:
    job_num=op.job
    op_num=op.op_num
    existed=False
    for m in range(machine_M):
        if (job_num in machines_jobs[m]):
            existed=True
            catch_latest_op=retrieve_last_operation(op,(operations_seq[:operations_seq.index(op)]).copy()) #from here we can take the machine and C
    if not existed:
        #print("the job didn't exist before in any machine, all good")
        #if the machine seq is empty
        if not machines[op.machine-1]:
            op.C=op.duration
            machines[op.machine-1].append(op)
            machines_jobs[op.machine-1].append(op.job)
        #if the machine seq is not empty
        else:
            op.C=machines[op.machine-1][::-1][0].C+op.duration #the end time for the last operation in the relevant machine + this operation duration time
            machines[op.machine-1].append(op)
            machines_jobs[op.machine-1].append(op.job)
    else: #existed=True
     #   print("this job of op: ", op)
       # print("existed before in: ",catch_latest_op)
        #print("the job existed before in a machine, we need to know which one and when did it end")
        catch_latest_op.machine
        catch_latest_op.C
        #the time the job finished in the past, determines when is the lowest time it can start in the next machine
        #if the machine seq is empty
        if not machines[op.machine-1]:
            op.C=catch_latest_op.C+op.duration
            machines[op.machine-1].append(op)
            machines_jobs[op.machine-1].append(op.job)
        #if the machine seq is not empty
        else:
            op.C=max(catch_latest_op.C,machines[op.machine-1][::-1][0].C)+op.duration 
            machines[op.machine-1].append(op)
            machines_jobs[op.machine-1].append(op.job) 
    print(op)
C=max([machines[m][::-1][0].C for m in range(machine_M)])
print("the maxspan C for this sequence is: ",C)

Job:3  Op:1  Machine:2  Duration: 1   C: 1 
Job:3  Op:2  Machine:1  Duration: 1   C: 2 
Job:1  Op:1  Machine:1  Duration: 2   C: 4 
Job:1  Op:2  Machine:2  Duration: 2   C: 6 
Job:2  Op:1  Machine:2  Duration: 3   C: 9 
Job:2  Op:2  Machine:1  Duration: 1   C: 10 
the maxspan C for this sequence is:  10


### customization section

In [19]:
#X as a list
def objective_function(particle_parameters):
    machines=list()
    machines_jobs=list()
    for m in range(machine_M):
        machines.append(list())
        machines_jobs.append(list())
    operations_seq=generate_operations_seq_from_particle_parameters(particle_parameters)
    for op in operations_seq:
        job_num=op.job
        op_num=op.op_num
        existed=False
        for m in range(machine_M):
            if (job_num in machines_jobs[m]):
                existed=True
                catch_latest_op=retrieve_last_operation(op,(operations_seq[:operations_seq.index(op)]).copy()) #from here we can take the machine and C
        if not existed:
            #print("the job didn't exist before in any machine, all good")
            #if the machine seq is empty
            if not machines[op.machine-1]:
                op.C=op.duration
                machines[op.machine-1].append(op)
                machines_jobs[op.machine-1].append(op.job)
            #if the machine seq is not empty
            else:
                op.C=machines[op.machine-1][::-1][0].C+op.duration #the end time for the last operation in the relevant machine + this operation duration time
                machines[op.machine-1].append(op)
                machines_jobs[op.machine-1].append(op.job)
        else: #existed=True
         #   print("this job of op: ", op)
           # print("existed before in: ",catch_latest_op)
            #print("the job existed before in a machine, we need to know which one and when did it end")
            catch_latest_op.machine
            catch_latest_op.C
            #the time the job finished in the past, determines when is the lowest time it can start in the next machine
            #if the machine seq is empty
            if not machines[op.machine-1]:
                op.C=catch_latest_op.C+op.duration
                machines[op.machine-1].append(op)
                machines_jobs[op.machine-1].append(op.job)
            #if the machine seq is not empty
            else:
                op.C=max(catch_latest_op.C,machines[op.machine-1][::-1][0].C)+op.duration 
                machines[op.machine-1].append(op)
                machines_jobs[op.machine-1].append(op.job) 
        print(op)
    C=max([machines[m][::-1][0].C for m in range(machine_M)])
    print("the maxspan C for this sequence is: ",C)
    return C

In [34]:
def generate_operations_seq_from_particle_parameters(list_A):
    list_A_con=[(sorted(list_A).index(element)+1) for element in list_A]
    order_jobs=[element%jobs_N +1 for element in list_A_con]
    operations_seq=list()
    for i in range(1,jobs_N+1):
        op_number=1
        for j in range(len(order_jobs)):
            if i==order_jobs[j]: 
               # print("j={} i={}, op_number={}".format(j,i,op_number))
                operations_seq.insert(j,Operation(i,op_number))
                op_number=op_number+1
    #for op in operations_seq:
     #   print(op)
    return operations_seq.copy()
    # NOW WE HAVE A LIST OF ALL OPERATIONS ! WE CAN CALCULATE THE "MAKESPAN"- i.e - the total duration of the sequence!
    

In [21]:
nv=6 #number of variables N jobs* M machines
mm=-1 #if min problem-->insert -1. if max problem insert mm=1
bounds=[(1,6),
       (1,6),
       (1,6),
       (1,6),
       (1,6),
       (1,6)]


In [22]:
particle_size=50 #num of particles
iterations=10 #max num of iterations
w=0.75 #inertia constant
c1=1 #cognative constant
c2=2 #social constant

### 

In [23]:
if mm==-1: #Min
    initial_fitness=float("inf")
if mm==1:
    initial_fitness=(-(float("inf"))) #Max

In [25]:
class Particle:
    def __init__(self,bounds):
        self.particle_position=[]
        self.particle_velocity=[]
        self.local_best_particle_position=[]
        self.fitness_particle_position=0  #initial obj func val of the best particle position
        self.fitness_local_best_particle_position=initial_fitness  #initial obj func val of the best particle position
        
        for i in range(nv):
            self.particle_position.append( random.uniform(bounds[i][0],bounds[i][1]) )
            self.particle_velocity.append( random.uniform(-1,1)  )
        for i in range(nv):
            if self.particle_position.count(self.particle_position[i])>1:
                self.particle_position[i]=self.particle_position[i]-0.00005
        if self.particle_position.count(6)>1:
            self.particle_position[self.particle_position.index(6)]=self.particle_position[self.particle_position.index(6)]-0.00005
            
        if self.particle_position.count(1)>1:
            self.particle_position[self.particle_position.index(1)]=self.particle_position[self.particle_position.index(1)]-0.00005            
            
    def evaluate(self,objective_function):
        self.fitness_particle_position=objective_function(self.particle_position)
        if mm==-1:
            if (self.fitness_particle_position < self.fitness_local_best_particle_position):
                self.local_best_particle_position=self.particle_position
                self.fitness_local_best_particle_position=self.fitness_particle_position
        if mm==1:
            if (self.fitness_particle_position > self.fitness_local_best_particle_position):
                self.local_best_particle_position=self.particle_position
                self.fitness_local_best_particle_position=self.fitness_particle_position
                
    def update_velocity(self,global_best_particle_position):
        for i in range(nv):
            r1=random.random()
            r2=random.random()
            cognitive_velocity=c1*r1*(self.local_best_particle_position[i]-self.particle_position[i])
            social_velocity=c2*r2*(global_best_particle_position[i]-self.particle_position[i])
            self.particle_velocity[i]=w*self.particle_velocity[i]+cognitive_velocity+social_velocity
    def update_position(self,bounds):
        for i in range(nv):
            self.particle_position[i]=self.particle_position[i]+self.particle_velocity[i]
            
            #check boundaries:
            if self.particle_position[i] > bounds[i][1]:
                self.particle_position[i]=bounds[i][1]
                
            if self.particle_position[i] < bounds[i][0]:
                self.particle_position[i]=bounds[i][0]
        while self.particle_position.count(6)>1:
            self.particle_position[self.particle_position.index(6)]=self.particle_position[self.particle_position.index(6)]-0.00005
            
        while self.particle_position.count(1)>1:
            self.particle_position[self.particle_position.index(1)]=self.particle_position[self.particle_position.index(1)]-0.00005                        
            
        for i in range(nv):
            while self.particle_position.count(self.particle_position[i])>1:
                self.particle_position[i]=self.particle_position[i]-0.00005

In [26]:
global_best_particle_position=[]
fitness_global_best_particle_position=initial_fitness

In [27]:
swarm_particle=[]

In [28]:
for i in range(particle_size):
    swarm_particle.append( Particle(bounds) )

In [29]:
A=[]

In [30]:
for i in range(iterations):
    for j in range(particle_size):
        print(swarm_particle[j].particle_position)
        swarm_particle[j].evaluate(objective_function)
        if mm==-1:
            if swarm_particle[j].fitness_particle_position<fitness_global_best_particle_position:
                global_best_particle_position=list(swarm_particle[j].particle_position)
                fitness_global_best_particle_position=float(swarm_particle[j].fitness_particle_position)
        if mm==1:
            if swarm_particle[j].fitness_particle_position>fitness_global_best_particle_position:
                global_best_particle_position=list(swarm_particle[j].particle_position)
                fitness_global_best_particle_position=float(swarm_particle[j].fitness_particle_position)
    for j in range(particle_size):
        swarm_particle[j].update_velocity(global_best_particle_position)
        swarm_particle[j].update_position(bounds)
        
    A.append(fitness_global_best_particle_position) #record for the best fitness

[4.049035000991123, 1.1906668790787518, 5.026499352663909, 4.7594833778666334, 3.868868934997856, 5.911554703429043]
Job:1  Op:1  Machine:1  Duration: 2   C: 0 
Job:2  Op:1  Machine:2  Duration: 3   C: 0 
Job:3  Op:1  Machine:2  Duration: 1   C: 0 
Job:1  Op:2  Machine:2  Duration: 2   C: 0 
Job:3  Op:2  Machine:1  Duration: 1   C: 0 
Job:2  Op:2  Machine:1  Duration: 1   C: 0 
Job:1  Op:1  Machine:1  Duration: 2   C: 2 
Job:2  Op:1  Machine:2  Duration: 3   C: 3 
Job:3  Op:1  Machine:2  Duration: 1   C: 4 
Job:1  Op:2  Machine:2  Duration: 2   C: 6 
Job:3  Op:2  Machine:1  Duration: 1   C: 5 
Job:2  Op:2  Machine:1  Duration: 1   C: 6 
the maxspan C for this sequence is:  6
[3.5531227734264528, 4.254591490635718, 3.733431635915784, 4.8674370406334715, 1.2570111438800198, 4.01626219420784]
Job:3  Op:1  Machine:2  Duration: 1   C: 0 
Job:3  Op:2  Machine:1  Duration: 1   C: 0 
Job:1  Op:1  Machine:1  Duration: 2   C: 0 
Job:1  Op:2  Machine:2  Duration: 2   C: 0 
Job:2  Op:1  Machine:2 

Job:2  Op:1  Machine:2  Duration: 3   C: 0 
Job:1  Op:1  Machine:1  Duration: 2   C: 0 
Job:3  Op:1  Machine:2  Duration: 1   C: 0 
Job:1  Op:2  Machine:2  Duration: 2   C: 0 
Job:3  Op:2  Machine:1  Duration: 1   C: 0 
Job:2  Op:2  Machine:1  Duration: 1   C: 0 
Job:2  Op:1  Machine:2  Duration: 3   C: 3 
Job:1  Op:1  Machine:1  Duration: 2   C: 2 
Job:3  Op:1  Machine:2  Duration: 1   C: 4 
Job:1  Op:2  Machine:2  Duration: 2   C: 6 
Job:3  Op:2  Machine:1  Duration: 1   C: 5 
Job:2  Op:2  Machine:1  Duration: 1   C: 6 
the maxspan C for this sequence is:  6
[5.89133993553904, 2.6381691504576783, 4.560591644349218, 5.974610227139655, 4.915521747969213, 2.512950125989392]
Job:3  Op:1  Machine:2  Duration: 1   C: 0 
Job:3  Op:2  Machine:1  Duration: 1   C: 0 
Job:1  Op:1  Machine:1  Duration: 2   C: 0 
Job:1  Op:2  Machine:2  Duration: 2   C: 0 
Job:2  Op:1  Machine:2  Duration: 3   C: 0 
Job:2  Op:2  Machine:1  Duration: 1   C: 0 
Job:3  Op:1  Machine:2  Duration: 1   C: 1 
Job:3  Op:

Job:1  Op:1  Machine:1  Duration: 2   C: 0 
Job:2  Op:1  Machine:2  Duration: 3   C: 0 
Job:3  Op:1  Machine:2  Duration: 1   C: 0 
Job:1  Op:2  Machine:2  Duration: 2   C: 0 
Job:2  Op:2  Machine:1  Duration: 1   C: 0 
Job:3  Op:2  Machine:1  Duration: 1   C: 0 
Job:1  Op:1  Machine:1  Duration: 2   C: 2 
Job:2  Op:1  Machine:2  Duration: 3   C: 3 
Job:3  Op:1  Machine:2  Duration: 1   C: 4 
Job:1  Op:2  Machine:2  Duration: 2   C: 6 
Job:2  Op:2  Machine:1  Duration: 1   C: 4 
Job:3  Op:2  Machine:1  Duration: 1   C: 5 
the maxspan C for this sequence is:  6
[1.4227863278961346, 1.9129777167331858, 2.5750877248814077, 5.626271594717536, 4.179714524574136, 2.6401718290085645]
Job:2  Op:1  Machine:2  Duration: 3   C: 0 
Job:3  Op:1  Machine:2  Duration: 1   C: 0 
Job:1  Op:1  Machine:1  Duration: 2   C: 0 
Job:1  Op:2  Machine:2  Duration: 2   C: 0 
Job:3  Op:2  Machine:1  Duration: 1   C: 0 
Job:2  Op:2  Machine:1  Duration: 1   C: 0 
Job:2  Op:1  Machine:2  Duration: 3   C: 3 
Job:3 

Job:3  Op:1  Machine:2  Duration: 1   C: 0 
Job:1  Op:1  Machine:1  Duration: 2   C: 0 
Job:2  Op:1  Machine:2  Duration: 3   C: 0 
Job:3  Op:2  Machine:1  Duration: 1   C: 0 
Job:2  Op:2  Machine:1  Duration: 1   C: 0 
Job:1  Op:2  Machine:2  Duration: 2   C: 0 
Job:3  Op:1  Machine:2  Duration: 1   C: 1 
Job:1  Op:1  Machine:1  Duration: 2   C: 2 
Job:2  Op:1  Machine:2  Duration: 3   C: 4 
Job:3  Op:2  Machine:1  Duration: 1   C: 3 
Job:2  Op:2  Machine:1  Duration: 1   C: 5 
Job:1  Op:2  Machine:2  Duration: 2   C: 6 
the maxspan C for this sequence is:  6
[3.9489632603009994, 2.1556872667087466, 4.0094059031462495, 5.417126084475451, 3.9793363660290972, 5.721521789069191]
Job:3  Op:1  Machine:2  Duration: 1   C: 0 
Job:1  Op:1  Machine:1  Duration: 2   C: 0 
Job:2  Op:1  Machine:2  Duration: 3   C: 0 
Job:3  Op:2  Machine:1  Duration: 1   C: 0 
Job:2  Op:2  Machine:1  Duration: 1   C: 0 
Job:1  Op:2  Machine:2  Duration: 2   C: 0 
Job:3  Op:1  Machine:2  Duration: 1   C: 1 
Job:1 

Job:1  Op:1  Machine:1  Duration: 2   C: 0 
Job:2  Op:1  Machine:2  Duration: 3   C: 0 
Job:3  Op:1  Machine:2  Duration: 1   C: 0 
Job:1  Op:2  Machine:2  Duration: 2   C: 0 
Job:2  Op:2  Machine:1  Duration: 1   C: 0 
Job:3  Op:2  Machine:1  Duration: 1   C: 0 
Job:1  Op:1  Machine:1  Duration: 2   C: 2 
Job:2  Op:1  Machine:2  Duration: 3   C: 3 
Job:3  Op:1  Machine:2  Duration: 1   C: 4 
Job:1  Op:2  Machine:2  Duration: 2   C: 6 
Job:2  Op:2  Machine:1  Duration: 1   C: 4 
Job:3  Op:2  Machine:1  Duration: 1   C: 5 
the maxspan C for this sequence is:  6
[5.087499798735113, 1, 3.771679819869221, 4.900481434601403, 2.659464742271574, 6]
Job:3  Op:1  Machine:2  Duration: 1   C: 0 
Job:1  Op:1  Machine:1  Duration: 2   C: 0 
Job:2  Op:1  Machine:2  Duration: 3   C: 0 
Job:1  Op:2  Machine:2  Duration: 2   C: 0 
Job:3  Op:2  Machine:1  Duration: 1   C: 0 
Job:2  Op:2  Machine:1  Duration: 1   C: 0 
Job:3  Op:1  Machine:2  Duration: 1   C: 1 
Job:1  Op:1  Machine:1  Duration: 2   C: 2

...

Job:1  Op:1  Machine:1  Duration: 2   C: 0 
Job:2  Op:1  Machine:2  Duration: 3   C: 0 
Job:3  Op:1  Machine:2  Duration: 1   C: 0 
Job:1  Op:2  Machine:2  Duration: 2   C: 0 
Job:3  Op:2  Machine:1  Duration: 1   C: 0 
Job:2  Op:2  Machine:1  Duration: 1   C: 0 
Job:1  Op:1  Machine:1  Duration: 2   C: 2 
Job:2  Op:1  Machine:2  Duration: 3   C: 3 
Job:3  Op:1  Machine:2  Duration: 1   C: 4 
Job:1  Op:2  Machine:2  Duration: 2   C: 6 
Job:3  Op:2  Machine:1  Duration: 1   C: 5 
Job:2  Op:2  Machine:1  Duration: 1   C: 6 
the maxspan C for this sequence is:  6
[3.862075801185151, 1.3329141511783, 4.472071190036263, 3.1390058428132654, 3.8443526068796072, 6]
Job:2  Op:1  Machine:2  Duration: 3   C: 0 
Job:2  Op:2  Machine:1  Duration: 1   C: 0 
Job:3  Op:1  Machine:2  Duration: 1   C: 0 
Job:3  Op:2  Machine:1  Duration: 1   C: 0 
Job:1  Op:1  Machine:1  Duration: 2   C: 0 
Job:1  Op:2  Machine:2  Duration: 2   C: 0 
Job:2  Op:1  Machine:2  Duration: 3   C: 3 
Job:2  Op:2  Machine:1  Du

Job:2  Op:1  Machine:2  Duration: 3   C: 0 
Job:2  Op:2  Machine:1  Duration: 1   C: 0 
Job:3  Op:1  Machine:2  Duration: 1   C: 0 
Job:1  Op:1  Machine:1  Duration: 2   C: 0 
Job:3  Op:2  Machine:1  Duration: 1   C: 0 
Job:1  Op:2  Machine:2  Duration: 2   C: 0 
Job:2  Op:1  Machine:2  Duration: 3   C: 3 
Job:2  Op:2  Machine:1  Duration: 1   C: 4 
Job:3  Op:1  Machine:2  Duration: 1   C: 4 
Job:1  Op:1  Machine:1  Duration: 2   C: 6 
Job:3  Op:2  Machine:1  Duration: 1   C: 7 
Job:1  Op:2  Machine:2  Duration: 2   C: 8 
the maxspan C for this sequence is:  8
[3.6863375819570505, 1.2583762805932515, 4.712480892967992, 4.4015909597526885, 2.2259205629641534, 6]
Job:1  Op:1  Machine:1  Duration: 2   C: 0 
Job:2  Op:1  Machine:2  Duration: 3   C: 0 
Job:3  Op:1  Machine:2  Duration: 1   C: 0 
Job:1  Op:2  Machine:2  Duration: 2   C: 0 
Job:3  Op:2  Machine:1  Duration: 1   C: 0 
Job:2  Op:2  Machine:1  Duration: 1   C: 0 
Job:1  Op:1  Machine:1  Duration: 2   C: 2 
Job:2  Op:1  Machine:2

Job:1  Op:1  Machine:1  Duration: 2   C: 0 
Job:2  Op:1  Machine:2  Duration: 3   C: 0 
Job:3  Op:1  Machine:2  Duration: 1   C: 0 
Job:1  Op:2  Machine:2  Duration: 2   C: 0 
Job:3  Op:2  Machine:1  Duration: 1   C: 0 
Job:2  Op:2  Machine:1  Duration: 1   C: 0 
Job:1  Op:1  Machine:1  Duration: 2   C: 2 
Job:2  Op:1  Machine:2  Duration: 3   C: 3 
Job:3  Op:1  Machine:2  Duration: 1   C: 4 
Job:1  Op:2  Machine:2  Duration: 2   C: 6 
Job:3  Op:2  Machine:1  Duration: 1   C: 5 
Job:2  Op:2  Machine:1  Duration: 1   C: 6 
the maxspan C for this sequence is:  6
[4.338490352750285, 1.0642450558416112, 4.810159993172973, 4.352050387243105, 4.213537812481372, 6]
Job:1  Op:1  Machine:1  Duration: 2   C: 0 
Job:2  Op:1  Machine:2  Duration: 3   C: 0 
Job:3  Op:1  Machine:2  Duration: 1   C: 0 
Job:1  Op:2  Machine:2  Duration: 2   C: 0 
Job:3  Op:2  Machine:1  Duration: 1   C: 0 
Job:2  Op:2  Machine:1  Duration: 1   C: 0 
Job:1  Op:1  Machine:1  Duration: 2   C: 2 
Job:2  Op:1  Machine:2  D

Job:3  Op:1  Machine:2  Duration: 1   C: 0 
Job:1  Op:1  Machine:1  Duration: 2   C: 0 
Job:3  Op:2  Machine:1  Duration: 1   C: 0 
Job:2  Op:1  Machine:2  Duration: 3   C: 0 
Job:1  Op:2  Machine:2  Duration: 2   C: 0 
Job:2  Op:2  Machine:1  Duration: 1   C: 0 
Job:3  Op:1  Machine:2  Duration: 1   C: 1 
Job:1  Op:1  Machine:1  Duration: 2   C: 2 
Job:3  Op:2  Machine:1  Duration: 1   C: 3 
Job:2  Op:1  Machine:2  Duration: 3   C: 4 
Job:1  Op:2  Machine:2  Duration: 2   C: 6 
Job:2  Op:2  Machine:1  Duration: 1   C: 5 
the maxspan C for this sequence is:  6
[4.282048271461428, 1.202408479077743, 5.651871410044148, 4.951984140613284, 4.141155623093979, 6]
Job:1  Op:1  Machine:1  Duration: 2   C: 0 
Job:2  Op:1  Machine:2  Duration: 3   C: 0 
Job:3  Op:1  Machine:2  Duration: 1   C: 0 
Job:1  Op:2  Machine:2  Duration: 2   C: 0 
Job:3  Op:2  Machine:1  Duration: 1   C: 0 
Job:2  Op:2  Machine:1  Duration: 1   C: 0 
Job:1  Op:1  Machine:1  Duration: 2   C: 2 
Job:2  Op:1  Machine:2  Du

Job:3  Op:1  Machine:2  Duration: 1   C: 0 
Job:1  Op:1  Machine:1  Duration: 2   C: 0 
Job:2  Op:1  Machine:2  Duration: 3   C: 0 
Job:3  Op:2  Machine:1  Duration: 1   C: 0 
Job:2  Op:2  Machine:1  Duration: 1   C: 0 
Job:1  Op:2  Machine:2  Duration: 2   C: 0 
Job:3  Op:1  Machine:2  Duration: 1   C: 1 
Job:1  Op:1  Machine:1  Duration: 2   C: 2 
Job:2  Op:1  Machine:2  Duration: 3   C: 4 
Job:3  Op:2  Machine:1  Duration: 1   C: 3 
Job:2  Op:2  Machine:1  Duration: 1   C: 5 
Job:1  Op:2  Machine:2  Duration: 2   C: 6 
the maxspan C for this sequence is:  6
[3.159595292126204, 1.0548357634420966, 5.644037158865269, 4.526986612310382, 4.364277931658475, 5.9051873317805805]
Job:3  Op:1  Machine:2  Duration: 1   C: 0 
Job:1  Op:1  Machine:1  Duration: 2   C: 0 
Job:3  Op:2  Machine:1  Duration: 1   C: 0 
Job:2  Op:1  Machine:2  Duration: 3   C: 0 
Job:1  Op:2  Machine:2  Duration: 2   C: 0 
Job:2  Op:2  Machine:1  Duration: 1   C: 0 
Job:3  Op:1  Machine:2  Duration: 1   C: 1 
Job:1  O

### answer:

In [35]:
print("optimal solution: ", global_best_particle_position)
for op in generate_operations_seq_from_particle_parameters(global_best_particle_position):
    print(op)
print("Obj Func Value: ", fitness_global_best_particle_position)

optimal solution:  [4.049035000991123, 1.1906668790787518, 5.026499352663909, 4.7594833778666334, 3.868868934997856, 5.911554703429043]
Job:1  Op:1  Machine:1  Duration: 2   C: 0 
Job:2  Op:1  Machine:2  Duration: 3   C: 0 
Job:3  Op:1  Machine:2  Duration: 1   C: 0 
Job:1  Op:2  Machine:2  Duration: 2   C: 0 
Job:3  Op:2  Machine:1  Duration: 1   C: 0 
Job:2  Op:2  Machine:1  Duration: 1   C: 0 
Obj Func Value:  6.0
